In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from tqdm import tqdm

# 설정 및 데이터 경로
config = {
    "Data": {
        "hla_embedding_path": "utils/1010basichla_embedding_diff1006.npy",  # HLA 임베딩 경로
        "epitope_embedding_path": "utils/1010basicepitope_embeddings1006.npy",  # Epitope 임베딩 경로
        "meta_data_path": "utils/1010basicencoded_sequences_metadata1006.csv",  # 메타 데이터 경로 (타겟 값 불러오기 위함)
        "val_size": 0.3,
    },
    "Train": {
        "num_epochs": 100,
        "learning_rate": 0.002,
        "batch_size": 128,
        "model_save_path": "model_checkpoint.pth",
        "early_stop_patience": 10  # Early Stopping patience 설정
    }
}

# 디바이스 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# HLA 임베딩과 Epitope 임베딩, 타겟 값 로드
hla_embeddings = np.load(config["Data"]["hla_embedding_path"], allow_pickle=True)
epitope_embeddings = np.load(config["Data"]["epitope_embedding_path"], allow_pickle=True)

# 메타데이터 로드 (타겟 값만 사용)
meta_data = pd.read_csv(config["Data"]["meta_data_path"])
targets = meta_data['target'].values.astype(np.float32)

# HLA와 Epitope 임베딩을 선형 결합 (단순히 합친다)
combined_embeddings = np.hstack((hla_embeddings, epitope_embeddings))

# 데이터 표준화
scaler = StandardScaler()
combined_embeddings = scaler.fit_transform(combined_embeddings)

# 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(combined_embeddings, targets, test_size=config["Data"]["val_size"], random_state=42)

# PyTorch Dataset 클래스 정의
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].unsqueeze(0), self.y[idx]  # (channels, length) 형태로 변환

# 데이터 로더 생성
train_loader = DataLoader(SequenceDataset(X_train, y_train), batch_size=config["Train"]["batch_size"], shuffle=True)
valid_loader = DataLoader(SequenceDataset(X_valid, y_valid), batch_size=config["Train"]["batch_size"], shuffle=False)

# 트랜스포머 기반 이진 분류 모델 정의
class BasicTransformerBinaryClassifier(nn.Module):
    def __init__(self, input_dim=2560, num_heads=4, num_layers=2, hidden_dim=512):
        super(BasicTransformerBinaryClassifier, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),  # 첫 번째 Linear Layer
            nn.BatchNorm1d(hidden_dim),        # Batch Normalization
            nn.ReLU(),                         # ReLU 활성화 함수
            nn.Dropout(0.5),                   # 50% 드롭아웃
    
            nn.Linear(hidden_dim, hidden_dim // 2),  # 추가된 히든 레이어
            nn.BatchNorm1d(hidden_dim // 2),         # Batch Normalization
            nn.LeakyReLU(0.1),                       # LeakyReLU 활성화 함수
            nn.Dropout(0.5),                         # 50% 드롭아웃
    
            nn.Linear(hidden_dim // 2, 1),  # 출력 레이어
            nn.Sigmoid()                    # Sigmoid 활성화 함수
        )

    def forward(self, x):
        x = self.transformer_encoder(x)  # 트랜스포머 인코더 통과
        x = x[:, 0, :]  # 첫 번째 위치의 출력을 사용하여 이진 분류
        return self.classifier(x).squeeze()

# 모델, 손실 함수, 옵티마이저 정의
model = BasicTransformerBinaryClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=config["Train"]["learning_rate"])

# 학습 및 검증 루프
num_epochs = config["Train"]["num_epochs"]
early_stop_patience = config["Train"]["early_stop_patience"]
best_valid_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    train_loader_tqdm = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}", leave=False)
    for batch_X, batch_y in train_loader_tqdm:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted_train = (outputs > 0.5).float()
        correct_train += (predicted_train == batch_y).sum().item()
        total_train += batch_y.size(0)
        train_loader_tqdm.set_postfix(loss=loss.item())  # 실시간 loss 출력

    train_accuracy = correct_train / total_train

    # 검증 루프
    model.eval()
    valid_loss = 0.0
    correct_valid = 0
    total_valid = 0

    valid_loader_tqdm = tqdm(valid_loader, desc=f"Validation Epoch {epoch + 1}/{num_epochs}", leave=False)
    with torch.no_grad():
        for batch_X, batch_y in valid_loader_tqdm:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            valid_loss += loss.item()
            predicted_valid = (outputs > 0.5).float()
            correct_valid += (predicted_valid == batch_y).sum().item()
            total_valid += batch_y.size(0)
            valid_loader_tqdm.set_postfix(loss=loss.item())  # 실시간 loss 출력

    valid_accuracy = correct_valid / total_valid

    # 학습 결과 출력
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}, "
          f"Valid Loss: {valid_loss / len(valid_loader):.4f}, "
          f"Valid Accuracy: {valid_accuracy:.4f}")

    # Early Stopping 체크
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        early_stop_counter = 0
        # 최적의 모델 저장
        torch.save(model.state_dict(), config["Train"]["model_save_path"])
        print(f"Validation loss improved, model saved to {config['Train']['model_save_path']}")
    else:
        early_stop_counter += 1
        print(f"Validation loss did not improve for {early_stop_counter} epochs.")

    # Early Stopping 조건 만족 시 학습 종료
    if early_stop_counter >= early_stop_patience:
        print("Early stopping triggered.")
        break

print("Training complete. Best model saved to:", config["Train"]["model_save_path"])


Using device: cuda


Epoch [1/100], Train Loss: 0.4697, Train Accuracy: 0.7699, Valid Loss: 0.4285, Valid Accuracy: 0.7894
Validation loss improved, model saved to model_checkpoint.pth


Epoch [2/100], Train Loss: 0.4152, Train Accuracy: 0.8040, Valid Loss: 0.4114, Valid Accuracy: 0.8013
Validation loss improved, model saved to model_checkpoint.pth


Epoch [3/100], Train Loss: 0.3897, Train Accuracy: 0.8199, Valid Loss: 0.3828, Valid Accuracy: 0.8232
Validation loss improved, model saved to model_checkpoint.pth


Epoch [4/100], Train Loss: 0.3695, Train Accuracy: 0.8312, Valid Loss: 0.3494, Valid Accuracy: 0.8445
Validation loss improved, model saved to model_checkpoint.pth


...

Epoch [39/100], Train Loss: 0.2232, Train Accuracy: 0.9081, Valid Loss: 0.2397, Valid Accuracy: 0.9031
Validation loss did not improve for 6 epochs.


Epoch [40/100], Train Loss: 0.2298, Train Accuracy: 0.9052, Valid Loss: 0.2329, Valid Accuracy: 0.9048
Validation loss did not improve for 7 epochs.


Epoch [41/100], Train Loss: 0.2296, Train Accuracy: 0.9061, Valid Loss: 0.2320, Valid Accuracy: 0.9047
Validation loss did not improve for 8 epochs.


Epoch [42/100], Train Loss: 0.2282, Train Accuracy: 0.9066, Valid Loss: 0.2356, Valid Accuracy: 0.9037
Validation loss did not improve for 9 epochs.


Epoch [43/100], Train Loss: 0.2276, Train Accuracy: 0.9063, Valid Loss: 0.2353, Valid Accuracy: 0.9043
Validation loss did not improve for 10 epochs.
Early stopping triggered.
Training complete. Best model saved to: model_checkpoint.pth


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from tqdm import tqdm

# 설정 및 데이터 경로
config = {
    "Data": {
        "hla_embedding_path": "utils/hla_embedding_diff1006.npy",  # HLA 임베딩 경로
        "epitope_embedding_path": "utils/epitope_embeddings1006.npy",  # Epitope 임베딩 경로
        "meta_data_path": "utils/encoded_sequences_metadata1006.csv",  # 메타 데이터 경로 (타겟 값 불러오기 위함)
        "val_size": 0.3,
    },
    "Train": {
        "num_epochs": 100,
        "learning_rate": 0.005,
        "batch_size": 256,
        "model_save_path": "model_checkpoint1007.pth",
        "early_stop_patience": 10  # Early Stopping patience 설정
    }
}

# 디바이스 설정 (GPU 사용 가능 시 GPU 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# HLA 임베딩과 Epitope 임베딩, 타겟 값 로드
hla_embeddings = np.load(config["Data"]["hla_embedding_path"], allow_pickle=True)
epitope_embeddings = np.load(config["Data"]["epitope_embedding_path"], allow_pickle=True)

# 메타데이터 로드 (타겟 값과 HLA 이름 불러오기)
meta_data = pd.read_csv(config["Data"]["meta_data_path"])
targets = meta_data['target'].values.astype(np.float32)
hla_names = meta_data['hla_name'].values

# HLA 임베딩을 HLA 이름에 맞게 매핑 (딕셔너리 형태로 저장)
hla_name_to_embedding = {hla_name: hla_embedding for hla_name, hla_embedding in zip(hla_names, hla_embeddings)}

# Epitope 임베딩에 맞는 HLA 임베딩을 결합
combined_embeddings = []
for i, row in meta_data.iterrows():
    hla_name = row['hla_name']
    epitope_embedding = epitope_embeddings[i]
    
    # HLA 이름에 맞는 HLA 임베딩을 찾음
    hla_embedding = hla_name_to_embedding.get(hla_name)
    
    # HLA 임베딩과 Epitope 임베딩을 결합
    combined_embedding = np.hstack((hla_embedding, epitope_embedding))
    combined_embeddings.append(combined_embedding)

combined_embeddings = np.array(combined_embeddings)

# 데이터 표준화
scaler = StandardScaler()
combined_embeddings = scaler.fit_transform(combined_embeddings)

# 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(combined_embeddings, targets, test_size=config["Data"]["val_size"], random_state=42)

# PyTorch Dataset 클래스 정의
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].unsqueeze(0), self.y[idx]  # (channels, length) 형태로 변환

# 데이터 로더 생성
train_loader = DataLoader(SequenceDataset(X_train, y_train), batch_size=config["Train"]["batch_size"], shuffle=True)
valid_loader = DataLoader(SequenceDataset(X_valid, y_valid), batch_size=config["Train"]["batch_size"], shuffle=False)

# 트랜스포머 기반 이진 분류 모델 정의
class BasicTransformerBinaryClassifier(nn.Module):
    def __init__(self, input_dim=2560, num_heads=8, num_layers=2, hidden_dim=512):
        super(BasicTransformerBinaryClassifier, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.classifier = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),  # 첫 번째 Linear Layer
            nn.BatchNorm1d(hidden_dim),        # Batch Normalization
            nn.ReLU(),                         # ReLU 활성화 함수
            nn.Dropout(0.5),                   # 50% 드롭아웃
    
            nn.Linear(hidden_dim, hidden_dim // 2),  # 추가된 히든 레이어
            nn.BatchNorm1d(hidden_dim // 2),         # Batch Normalization
            nn.LeakyReLU(0.1),                       # LeakyReLU 활성화 함수
            nn.Dropout(0.5),                         # 50% 드롭아웃
    
            nn.Linear(hidden_dim // 2, 1),  # 출력 레이어
            nn.Sigmoid()                    # Sigmoid 활성화 함수
        )

    def forward(self, x):
        x = self.transformer_encoder(x)  # 트랜스포머 인코더 통과
        x = x[:, 0, :]  # 첫 번째 위치의 출력을 사용하여 이진 분류
        return self.classifier(x).squeeze()

# 모델, 손실 함수, 옵티마이저 정의
model = BasicTransformerBinaryClassifier().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=config["Train"]["learning_rate"])

# 학습 및 검증 루프
num_epochs = config["Train"]["num_epochs"]
early_stop_patience = config["Train"]["early_stop_patience"]
best_valid_loss = float('inf')
early_stop_counter = 0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    correct_train = 0
    total_train = 0

    train_loader_tqdm = tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{num_epochs}", leave=False)
    for batch_X, batch_y in train_loader_tqdm:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        predicted_train = (outputs > 0.5).float()
        correct_train += (predicted_train == batch_y).sum().item()
        total_train += batch_y.size(0)
        train_loader_tqdm.set_postfix(loss=loss.item())  # 실시간 loss 출력

    train_accuracy = correct_train / total_train

    # 검증 루프
    model.eval()
    valid_loss = 0.0
    correct_valid = 0
    total_valid = 0

    valid_loader_tqdm = tqdm(valid_loader, desc=f"Validation Epoch {epoch + 1}/{num_epochs}", leave=False)
    with torch.no_grad():
        for batch_X, batch_y in valid_loader_tqdm:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            valid_loss += loss.item()
            predicted_valid = (outputs > 0.5).float()
            correct_valid += (predicted_valid == batch_y).sum().item()
            total_valid += batch_y.size(0)
            valid_loader_tqdm.set_postfix(loss=loss.item())  # 실시간 loss 출력

    valid_accuracy = correct_valid / total_valid

    # 학습 결과 출력
    print(f"Epoch [{epoch + 1}/{num_epochs}], "
          f"Train Loss: {train_loss / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}, "
          f"Valid Loss: {valid_loss / len(valid_loader):.4f}, "
          f"Valid Accuracy: {valid_accuracy:.4f}")

    # Early Stopping 체크
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        early_stop_counter = 0
        # 최적의 모델 저장
        torch.save(model.state_dict(), config["Train"]["model_save_path"])
        print(f"Validation loss improved, model saved to {config['Train']['model_save_path']}")
    else:
        early_stop_counter += 1
        print(f"Validation loss did not improve for {early_stop_counter} epochs.")

    # Early Stopping 조건 만족 시 학습 종료
    if early_stop_counter >= early_stop_patience:
        print("Early stopping triggered.")
        break

print("Training complete. Best model saved to:", config["Train"]["model_save_path"])


Using device: cuda


Epoch [1/100], Train Loss: 0.4562, Train Accuracy: 0.7684, Valid Loss: 0.3994, Valid Accuracy: 0.8153
Validation loss improved, model saved to model_checkpoint1007.pth


Epoch [2/100], Train Loss: 0.3815, Train Accuracy: 0.8330, Valid Loss: 0.3997, Valid Accuracy: 0.8164
Validation loss did not improve for 1 epochs.


Epoch [3/100], Train Loss: 0.3544, Train Accuracy: 0.8486, Valid Loss: 0.3461, Valid Accuracy: 0.8519
Validation loss improved, model saved to model_checkpoint1007.pth


Epoch [4/100], Train Loss: 0.3389, Train Accuracy: 0.8568, Valid Loss: 0.3271, Valid Accuracy: 0.8610
Validation loss improved, model saved to model_checkpoint1007.pth


...

Epoch [61/100], Train Loss: 0.2034, Train Accuracy: 0.9181, Valid Loss: 0.2329, Valid Accuracy: 0.9054
Validation loss did not improve for 6 epochs.


Epoch [62/100], Train Loss: 0.2027, Train Accuracy: 0.9190, Valid Loss: 0.2374, Valid Accuracy: 0.9023
Validation loss did not improve for 7 epochs.


Epoch [63/100], Train Loss: 0.2014, Train Accuracy: 0.9190, Valid Loss: 0.2353, Valid Accuracy: 0.9041
Validation loss did not improve for 8 epochs.


Epoch [64/100], Train Loss: 0.2012, Train Accuracy: 0.9197, Valid Loss: 0.2390, Valid Accuracy: 0.9023
Validation loss did not improve for 9 epochs.


Epoch [65/100], Train Loss: 0.2012, Train Accuracy: 0.9201, Valid Loss: 0.2344, Valid Accuracy: 0.9035
Validation loss did not improve for 10 epochs.
Early stopping triggered.
Training complete. Best model saved to: model_checkpoint1007.pth
